In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 


In [2]:
base_path = 'E:\\indoor-location-navigation\\'

In [3]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv(base_path + 'sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [4]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

In [21]:


for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train\\'+building+'\\*')))
    print(building)
    wifi = list()
    for folder in folders:
        print(folder)
        floor = floor_map[folder.split('\\')[-1]]
        print(floor)
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 0].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
E:\indoor-location-navigation\train\5a0546857ecc773753327266\B1
-1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F1
0
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F2
1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F3
2
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F4
3
3397
5c3c44b80379370013e0fd2b
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\B1
-1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F1
0
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F2
1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F3
2
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F4
3
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F5
4
3063
5d27075f03f801723c2e360f
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\B1
-1
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\F1
0
E:\indoor-location-navigation\train\5d27075f03f801723c

3535
5da958dd46f8266d0737457b
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\B1
-1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F1
0
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F2
1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F3
2
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F4
3
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F5
4
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F6
5
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F7
6
3499
5dbc1d84c1eb61796cf7c010
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\B1
-1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F2
1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F3
2
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F4
3
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F5
4
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F6
5
E:\indoor-loca

In [22]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

In [5]:
with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [24]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('\\')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            wifi = list()
            waypoint = list()
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)

            df = pd.DataFrame(np.array(wifi))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                # calculating x and y for wifi block
                calculated_coords = True
                
                if len(dists)>1:
                    second_nearest_wp_index = np.argsort(dists)[1]
                else:
                    calculated_coords = False
                    
                # need to define which time is in future
                if calculated_coords and int(waypoint[nearest_wp_index][0])> int(waypoint[second_nearest_wp_index][0]):
                    timestamp_future = int(waypoint[nearest_wp_index][0])
                    timestamp_past = int(waypoint[second_nearest_wp_index][0])
                    x_future = float(waypoint[nearest_wp_index][2])
                    y_future = float(waypoint[nearest_wp_index][3])
                    x_past = float(waypoint[second_nearest_wp_index][2])
                    y_past = float(waypoint[second_nearest_wp_index][3])
                elif calculated_coords:
                    timestamp_past = int(waypoint[nearest_wp_index][0])
                    timestamp_future = int(waypoint[second_nearest_wp_index][0])
                    x_past = float(waypoint[nearest_wp_index][2])
                    y_past = float(waypoint[nearest_wp_index][3])
                    x_future = float(waypoint[second_nearest_wp_index][2])
                    y_future = float(waypoint[second_nearest_wp_index][3])
                
                
                if calculated_coords and timestamp_past < int(gid) < timestamp_future: #Timestamp of wifi block between calculated timestamps
                    time_period =  timestamp_future - timestamp_past
                    x_inc_per_ms = (x_future - x_past) / time_period
                    y_inc_per_ms = (y_future - y_past) / time_period
                    x_calc = x_past + (int(gid)-timestamp_past)/time_period * x_inc_per_ms 
                    y_calc = y_past + (int(gid)-timestamp_past)/time_period * y_inc_per_ms
                    #print('Calculated coords are: ', x_calc, y_calc, 'floor ', floor, 'path ', file.split('\\')[-1].split('.')[0])
                    #print('Timestamps are: ', timestamp_past, int(gid), timestamp_future )
                else:
                    calculated_coords = False
                    # print("Calculated coords = False")
                    # print("Timestamps are: ", timestamp_past, int(gid), timestamp_future )
                    
                    
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).replace(np.nan, -999).T
                feat["x"] = x_calc if calculated_coords else float(waypoint[nearest_wp_index][2])
                feat["y"] = y_calc if calculated_coords else float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('\\')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [ ]:
ssubm_building_g = ssubm_df.groupby(0) #Grouped by building
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
#     print(gid0, 'gid0')
#     print(g0, 'g0')
#     print()
    for gid,g in g0.groupby(1):
        # get all wifi time locations, 
#         print(gid, 'gid')
#         print(g, 'g')
#         print()
        with open(os.path.join(base_path, 'test\\' + g.iloc[0,1] + '.txt'), encoding="utf8") as f:
            txt = f.readlines()

        wifi = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)

        wifi_df = pd.DataFrame(wifi) #All wifi points for certain path
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist()) #Timestamps for wifi for certain path
        #print(wifi_points)
        for timepoint in g.iloc[:,2].tolist():
            deltas = (wifi_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df

In [6]:
import xgboost as xgb

In [7]:
feature_path = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\'

In [8]:
train_files = sorted(glob.glob(os.path.join(feature_path, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_path, '*_test.csv')))

In [9]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    
    data = data.sample(frac=1)
    x_train = data.iloc[:int(len(data)*0.75),:-4]
    y_trainy = data.iloc[:int(len(data)*0.75),-3]
    y_trainx = data.iloc[:int(len(data)*0.75),-4]
    y_trainf = data.iloc[:int(len(data)*0.75),-2] + 2 #To make floors classes from 0 to 10
    
    x_test = data.iloc[int(len(data)*0.75):,:-4]
    y_testy = data.iloc[int(len(data)*0.75):,-3]
    y_testx = data.iloc[int(len(data)*0.75):,-4]
    y_testf = data.iloc[int(len(data)*0.75):,-2] + 2 #To make floors classes from 0 to 10
    
    
    dtrainX = xgb.DMatrix(x_train, label=y_trainx)
    dtestX = xgb.DMatrix(x_test, label=y_testx)
    
    dtrainY = xgb.DMatrix(x_train, label=y_trainy)
    dtestY = xgb.DMatrix(x_test, label=y_testy)
    
    dtrainF = xgb.DMatrix(x_train, label=y_trainf)
    dtestF = xgb.DMatrix(x_test, label=y_testf)
    
    paramsXY = {
    # Parameters that we are going to tune.
    'max_depth':50,
    'min_child_weight': 0.5,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.6,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    'booster':'gbtree',
    'random_state':42,
    'verbosity':1,
    'n_jobs':-1
    }
    
    paramsF = {
    # Parameters that we are going to tune.
    'max_depth':50,
    'min_child_weight': 0.5,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.6,
    # Other parameters
    'objective':'multi:softmax',
    'eval_metric':'merror',
    'num_class':11,
    'booster':'gbtree',
    'random_state':42,
    'verbosity':1,
    'n_jobs':-1
    }
    
    num_boost_round=999
    
    xgb_rX = xgb.train(
    paramsXY,
    dtrainX,
    num_boost_round=num_boost_round,
    evals=[(dtestX, "Test")],
    early_stopping_rounds=50
    )
    
    xgb_rY = xgb.train(
    paramsXY,
    dtrainY,
    num_boost_round=num_boost_round,
    evals=[(dtestY, "Test")],
    early_stopping_rounds=50
    )
    
    xgb_cF = xgb.train(
    paramsF,
    dtrainF,
    num_boost_round=num_boost_round,
    evals=[(dtestF, "Test")],
    early_stopping_rounds=50
    )
    
#     modely = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modely.fit(x_train, y_trainy)

#     modelx = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modelx.fit(x_train, y_trainx)

#     modelf = lgb.LGBMClassifier(
#         n_estimators=125, num_leaves=90)
#     modelf.fit(x_train, y_trainf)
    
    test_predsx = xgb_rX.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsy = xgb_rY.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsf = xgb_cF.predict(xgb.DMatrix(test_data.iloc[:,:-1]+2))
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns[1:]
    test_preds.index = test_data["site_path_timestamp"]
    
    test_preds["floor"] = test_preds["floor"].astype(int)-2 #Returning floor classes back to original
    
    #Post-processing should go here. Idea is to snap to waypoint if the prediction is to close.
    #*************************
    #Generating dataframe with unique waypoints
    
    #data.drop_duplicates(subset=['x', 'y', 'f'], inplace=True)
    
    #*********End of post processing**********
    
    
    
    
    predictions.append(test_preds)

[0]	Test-rmse:90.69663
[1]	Test-rmse:72.91864
[2]	Test-rmse:58.62050
[3]	Test-rmse:47.20825
[4]	Test-rmse:38.03421
[5]	Test-rmse:30.74859
[6]	Test-rmse:24.93045
[7]	Test-rmse:20.29641
[8]	Test-rmse:16.63804
[9]	Test-rmse:13.74195
[10]	Test-rmse:11.46953
[11]	Test-rmse:9.71472
[12]	Test-rmse:8.32290
[13]	Test-rmse:7.29018
[14]	Test-rmse:6.52710
[15]	Test-rmse:5.95079
[16]	Test-rmse:5.51603
[17]	Test-rmse:5.21564
[18]	Test-rmse:4.98382
[19]	Test-rmse:4.82160
[20]	Test-rmse:4.70207
[21]	Test-rmse:4.60796
[22]	Test-rmse:4.54766
[23]	Test-rmse:4.50600
[24]	Test-rmse:4.47414
[25]	Test-rmse:4.44852
[26]	Test-rmse:4.42990
[27]	Test-rmse:4.41369
[28]	Test-rmse:4.39754
[29]	Test-rmse:4.38853
[30]	Test-rmse:4.38063
[31]	Test-rmse:4.37417
[32]	Test-rmse:4.36983
[33]	Test-rmse:4.36606
[34]	Test-rmse:4.36267
[35]	Test-rmse:4.36025
[36]	Test-rmse:4.35557
[37]	Test-rmse:4.35305
[38]	Test-rmse:4.35189
[39]	Test-rmse:4.35048
[40]	Test-rmse:4.34852
[41]	Test-rmse:4.34604
[42]	Test-rmse:4.34471
[43]	Test-

[64]	Test-rmse:3.16470
[65]	Test-rmse:3.16466
[66]	Test-rmse:3.16463
[67]	Test-rmse:3.16458
[68]	Test-rmse:3.16456
[69]	Test-rmse:3.16454
[70]	Test-rmse:3.16451
[71]	Test-rmse:3.16446
[72]	Test-rmse:3.16447
[73]	Test-rmse:3.16446
[74]	Test-rmse:3.16443
[75]	Test-rmse:3.16441
[76]	Test-rmse:3.16441
[77]	Test-rmse:3.16438
[78]	Test-rmse:3.16437
[79]	Test-rmse:3.16437
[80]	Test-rmse:3.16436
[81]	Test-rmse:3.16433
[82]	Test-rmse:3.16433
[83]	Test-rmse:3.16432
[84]	Test-rmse:3.16430
[85]	Test-rmse:3.16428
[86]	Test-rmse:3.16426
[87]	Test-rmse:3.16422
[88]	Test-rmse:3.16422
[89]	Test-rmse:3.16421
[90]	Test-rmse:3.16418
[91]	Test-rmse:3.16418
[92]	Test-rmse:3.16416
[93]	Test-rmse:3.16417
[94]	Test-rmse:3.16417
[95]	Test-rmse:3.16416
[96]	Test-rmse:3.16415
[97]	Test-rmse:3.16415
[98]	Test-rmse:3.16414
[99]	Test-rmse:3.16413
[100]	Test-rmse:3.16414
[101]	Test-rmse:3.16413
[102]	Test-rmse:3.16413
[103]	Test-rmse:3.16412
[104]	Test-rmse:3.16411
[105]	Test-rmse:3.16410
[106]	Test-rmse:3.16409
[107

[154]	Test-rmse:5.49373
[155]	Test-rmse:5.49372
[156]	Test-rmse:5.49372
[157]	Test-rmse:5.49372
[158]	Test-rmse:5.49372
[159]	Test-rmse:5.49372
[160]	Test-rmse:5.49372
[161]	Test-rmse:5.49373
[162]	Test-rmse:5.49372
[163]	Test-rmse:5.49372
[164]	Test-rmse:5.49372
[165]	Test-rmse:5.49372
[166]	Test-rmse:5.49372
[167]	Test-rmse:5.49372
[168]	Test-rmse:5.49373
[169]	Test-rmse:5.49372
[170]	Test-rmse:5.49372
[171]	Test-rmse:5.49372
[172]	Test-rmse:5.49372
[173]	Test-rmse:5.49372
[174]	Test-rmse:5.49372
[175]	Test-rmse:5.49372
[176]	Test-rmse:5.49372
[177]	Test-rmse:5.49372
[178]	Test-rmse:5.49372
[179]	Test-rmse:5.49372
[180]	Test-rmse:5.49372
[181]	Test-rmse:5.49372
[182]	Test-rmse:5.49372
[183]	Test-rmse:5.49372
[184]	Test-rmse:5.49372
[185]	Test-rmse:5.49372
[186]	Test-rmse:5.49372
[187]	Test-rmse:5.49372
[188]	Test-rmse:5.49372
[189]	Test-rmse:5.49372
[190]	Test-rmse:5.49372
[191]	Test-rmse:5.49372
[192]	Test-rmse:5.49372
[193]	Test-rmse:5.49372
[194]	Test-rmse:5.49372
[195]	Test-rmse:

[24]	Test-rmse:4.24848
[25]	Test-rmse:4.20711
[26]	Test-rmse:4.17419
[27]	Test-rmse:4.14900
[28]	Test-rmse:4.13329
[29]	Test-rmse:4.11981
[30]	Test-rmse:4.11008
[31]	Test-rmse:4.10113
[32]	Test-rmse:4.09559
[33]	Test-rmse:4.09041
[34]	Test-rmse:4.08623
[35]	Test-rmse:4.08347
[36]	Test-rmse:4.08140
[37]	Test-rmse:4.07982
[38]	Test-rmse:4.07835
[39]	Test-rmse:4.07658
[40]	Test-rmse:4.07527
[41]	Test-rmse:4.07429
[42]	Test-rmse:4.07388
[43]	Test-rmse:4.07301
[44]	Test-rmse:4.07179
[45]	Test-rmse:4.07121
[46]	Test-rmse:4.07056
[47]	Test-rmse:4.07015
[48]	Test-rmse:4.06964
[49]	Test-rmse:4.06954
[50]	Test-rmse:4.06926
[51]	Test-rmse:4.06921
[52]	Test-rmse:4.06893
[53]	Test-rmse:4.06888
[54]	Test-rmse:4.06878
[55]	Test-rmse:4.06869
[56]	Test-rmse:4.06868
[57]	Test-rmse:4.06867
[58]	Test-rmse:4.06858
[59]	Test-rmse:4.06856
[60]	Test-rmse:4.06844
[61]	Test-rmse:4.06834
[62]	Test-rmse:4.06830
[63]	Test-rmse:4.06813
[64]	Test-rmse:4.06811
[65]	Test-rmse:4.06790
[66]	Test-rmse:4.06791
[67]	Test-r

[369]	Test-rmse:4.06589
[370]	Test-rmse:4.06589
[0]	Test-rmse:95.73396
[1]	Test-rmse:76.84735
[2]	Test-rmse:61.71740
[3]	Test-rmse:49.61722
[4]	Test-rmse:39.94477
[5]	Test-rmse:32.20097
[6]	Test-rmse:26.02112
[7]	Test-rmse:21.08470
[8]	Test-rmse:17.16778
[9]	Test-rmse:14.04868
[10]	Test-rmse:11.58651
[11]	Test-rmse:9.64245
[12]	Test-rmse:8.10818
[13]	Test-rmse:6.92231
[14]	Test-rmse:6.01491
[15]	Test-rmse:5.31021
[16]	Test-rmse:4.78045
[17]	Test-rmse:4.38664
[18]	Test-rmse:4.09480
[19]	Test-rmse:3.88240
[20]	Test-rmse:3.72537
[21]	Test-rmse:3.61376
[22]	Test-rmse:3.52813
[23]	Test-rmse:3.46899
[24]	Test-rmse:3.42670
[25]	Test-rmse:3.39163
[26]	Test-rmse:3.36435
[27]	Test-rmse:3.34436
[28]	Test-rmse:3.33096
[29]	Test-rmse:3.31718
[30]	Test-rmse:3.30723
[31]	Test-rmse:3.29977
[32]	Test-rmse:3.29335
[33]	Test-rmse:3.28832
[34]	Test-rmse:3.28324
[35]	Test-rmse:3.28012
[36]	Test-rmse:3.27632
[37]	Test-rmse:3.27347
[38]	Test-rmse:3.27151
[39]	Test-rmse:3.27041
[40]	Test-rmse:3.26940
[41]	Tes

...

[49]	Test-rmse:3.97884
[50]	Test-rmse:3.97848
[51]	Test-rmse:3.97826
[52]	Test-rmse:3.97800
[53]	Test-rmse:3.97736
[54]	Test-rmse:3.97720
[55]	Test-rmse:3.97700
[56]	Test-rmse:3.97672
[57]	Test-rmse:3.97653
[58]	Test-rmse:3.97651
[59]	Test-rmse:3.97638
[60]	Test-rmse:3.97622
[61]	Test-rmse:3.97605
[62]	Test-rmse:3.97602
[63]	Test-rmse:3.97574
[64]	Test-rmse:3.97559
[65]	Test-rmse:3.97549
[66]	Test-rmse:3.97529
[67]	Test-rmse:3.97519
[68]	Test-rmse:3.97508
[69]	Test-rmse:3.97492
[70]	Test-rmse:3.97454
[71]	Test-rmse:3.97447
[72]	Test-rmse:3.97441
[73]	Test-rmse:3.97431
[74]	Test-rmse:3.97427
[75]	Test-rmse:3.97407
[76]	Test-rmse:3.97398
[77]	Test-rmse:3.97374
[78]	Test-rmse:3.97348
[79]	Test-rmse:3.97340
[80]	Test-rmse:3.97333
[81]	Test-rmse:3.97337
[82]	Test-rmse:3.97324
[83]	Test-rmse:3.97320
[84]	Test-rmse:3.97307
[85]	Test-rmse:3.97303
[86]	Test-rmse:3.97295
[87]	Test-rmse:3.97293
[88]	Test-rmse:3.97294
[89]	Test-rmse:3.97289
[90]	Test-rmse:3.97288
[91]	Test-rmse:3.97285
[92]	Test-r

[69]	Test-rmse:4.17196
[70]	Test-rmse:4.17179
[71]	Test-rmse:4.17148
[72]	Test-rmse:4.17143
[73]	Test-rmse:4.17139
[74]	Test-rmse:4.17142
[75]	Test-rmse:4.17134
[76]	Test-rmse:4.17129
[77]	Test-rmse:4.17122
[78]	Test-rmse:4.17110
[79]	Test-rmse:4.17103
[80]	Test-rmse:4.17100
[81]	Test-rmse:4.17100
[82]	Test-rmse:4.17097
[83]	Test-rmse:4.17080
[84]	Test-rmse:4.17080
[85]	Test-rmse:4.17074
[86]	Test-rmse:4.17054
[87]	Test-rmse:4.17045
[88]	Test-rmse:4.17036
[89]	Test-rmse:4.17033
[90]	Test-rmse:4.17027
[91]	Test-rmse:4.17021
[92]	Test-rmse:4.17010
[93]	Test-rmse:4.17008
[94]	Test-rmse:4.17002
[95]	Test-rmse:4.16996
[96]	Test-rmse:4.16995
[97]	Test-rmse:4.16996
[98]	Test-rmse:4.16994
[99]	Test-rmse:4.16994
[100]	Test-rmse:4.16990
[101]	Test-rmse:4.16974
[102]	Test-rmse:4.16969
[103]	Test-rmse:4.16969
[104]	Test-rmse:4.16966
[105]	Test-rmse:4.16967
[106]	Test-rmse:4.16961
[107]	Test-rmse:4.16959
[108]	Test-rmse:4.16958
[109]	Test-rmse:4.16954
[110]	Test-rmse:4.16951
[111]	Test-rmse:4.16944

[4]	Test-rmse:56.03015
[5]	Test-rmse:45.11687
[6]	Test-rmse:36.36543
[7]	Test-rmse:29.40440
[8]	Test-rmse:23.86774
[9]	Test-rmse:19.45599
[10]	Test-rmse:15.97589
[11]	Test-rmse:13.22105
[12]	Test-rmse:11.05670
[13]	Test-rmse:9.37102
[14]	Test-rmse:8.07493
[15]	Test-rmse:7.07305
[16]	Test-rmse:6.33782
[17]	Test-rmse:5.77919
[18]	Test-rmse:5.36172
[19]	Test-rmse:5.05735
[20]	Test-rmse:4.84722
[21]	Test-rmse:4.69489
[22]	Test-rmse:4.57961
[23]	Test-rmse:4.49781
[24]	Test-rmse:4.43142
[25]	Test-rmse:4.38597
[26]	Test-rmse:4.35478
[27]	Test-rmse:4.32594
[28]	Test-rmse:4.30612
[29]	Test-rmse:4.29220
[30]	Test-rmse:4.28211
[31]	Test-rmse:4.27107
[32]	Test-rmse:4.26181
[33]	Test-rmse:4.25784
[34]	Test-rmse:4.25192
[35]	Test-rmse:4.24852
[36]	Test-rmse:4.24587
[37]	Test-rmse:4.24454
[38]	Test-rmse:4.24296
[39]	Test-rmse:4.24077
[40]	Test-rmse:4.23995
[41]	Test-rmse:4.23724
[42]	Test-rmse:4.23651
[43]	Test-rmse:4.23518
[44]	Test-rmse:4.23416
[45]	Test-rmse:4.23314
[46]	Test-rmse:4.23259
[47]	Tes

[350]	Test-rmse:4.21129
[351]	Test-rmse:4.21129
[352]	Test-rmse:4.21129
[353]	Test-rmse:4.21129
[354]	Test-rmse:4.21129
[355]	Test-rmse:4.21129
[356]	Test-rmse:4.21129
[357]	Test-rmse:4.21129
[358]	Test-rmse:4.21129
[359]	Test-rmse:4.21129
[360]	Test-rmse:4.21129
[361]	Test-rmse:4.21129
[362]	Test-rmse:4.21129
[363]	Test-rmse:4.21129
[364]	Test-rmse:4.21129
[365]	Test-rmse:4.21129
[366]	Test-rmse:4.21129
[367]	Test-rmse:4.21129
[368]	Test-rmse:4.21129
[369]	Test-rmse:4.21129
[370]	Test-rmse:4.21129
[371]	Test-rmse:4.21129
[372]	Test-rmse:4.21129
[373]	Test-rmse:4.21129
[374]	Test-rmse:4.21129
[375]	Test-rmse:4.21129
[376]	Test-rmse:4.21129
[377]	Test-rmse:4.21129
[378]	Test-rmse:4.21129
[379]	Test-rmse:4.21129
[380]	Test-rmse:4.21129
[381]	Test-rmse:4.21129
[382]	Test-rmse:4.21129
[383]	Test-rmse:4.21129
[384]	Test-rmse:4.21129
[385]	Test-rmse:4.21129
[386]	Test-rmse:4.21129
[387]	Test-rmse:4.21129
[388]	Test-rmse:4.21129
[389]	Test-rmse:4.21129
[390]	Test-rmse:4.21129
[391]	Test-rmse:

[294]	Test-rmse:3.67561
[295]	Test-rmse:3.67561
[296]	Test-rmse:3.67561
[297]	Test-rmse:3.67561
[298]	Test-rmse:3.67560
[299]	Test-rmse:3.67560
[300]	Test-rmse:3.67560
[301]	Test-rmse:3.67560
[302]	Test-rmse:3.67560
[303]	Test-rmse:3.67559
[304]	Test-rmse:3.67559
[305]	Test-rmse:3.67559
[306]	Test-rmse:3.67559
[307]	Test-rmse:3.67559
[308]	Test-rmse:3.67559
[309]	Test-rmse:3.67559
[310]	Test-rmse:3.67559
[311]	Test-rmse:3.67559
[312]	Test-rmse:3.67559
[313]	Test-rmse:3.67559
[314]	Test-rmse:3.67559
[315]	Test-rmse:3.67559
[316]	Test-rmse:3.67559
[317]	Test-rmse:3.67559
[318]	Test-rmse:3.67559
[319]	Test-rmse:3.67558
[320]	Test-rmse:3.67558
[321]	Test-rmse:3.67558
[322]	Test-rmse:3.67559
[323]	Test-rmse:3.67559
[324]	Test-rmse:3.67559
[325]	Test-rmse:3.67559
[326]	Test-rmse:3.67559
[327]	Test-rmse:3.67559
[328]	Test-rmse:3.67559
[329]	Test-rmse:3.67558
[330]	Test-rmse:3.67559
[331]	Test-rmse:3.67558
[332]	Test-rmse:3.67558
[333]	Test-rmse:3.67557
[334]	Test-rmse:3.67557
[335]	Test-rmse:

In [10]:
# generate prediction file 
all_preds = pd.concat(predictions)


In [11]:
all_preds.reset_index(inplace=True)

In [12]:
all_preds = all_preds.reindex(ssubm.index)


In [13]:
all_preds.to_csv('submission.csv', index=False)

In [14]:
all_preds

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.939186,94.909569
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.923904,94.103546
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.637100,103.851868
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.419456,103.945190
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.712601,107.202095
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,216.940613,95.570831
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,213.767960,101.750237
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,207.115051,104.301048
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,204.074188,112.260551


,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.939186,94.909569
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.923904,94.103546
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.637100,103.851868
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.419456,103.945190
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.712601,107.202095
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,216.940613,95.570831
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,213.767960,101.750237
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,207.115051,104.301048
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,204.074188,112.260551
